#### Homework
In this homework, we'll deploy the Straight vs Curly Hair Type model we trained in the
[previous homework](../08-deep-learning/homework.md).

Download the model from here:

https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/model_2024_hairstyle.keras

## Question 1

Now convert this model from Keras to TF-Lite format.

What's the size of the **converted** model?

* 27 Mb
* 43 Mb
* 77 Mb  (correct)
* 127 Mb


In [5]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
tf.__version__

'2.17.0'

In [17]:
!wget https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/model_2024_hairstyle.keras


In [7]:
model = keras.models.load_model("model_2024_hairstyle.keras")


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open("model_2024_hairstyle.tflite", "wb") as f_out:
    f_out.write(tflite_model)


In [9]:
import os

def get_file_size(filename):
    size_bytes = os.path.getsize(filename)    
    size_mb = size_bytes / (1024 * 1024)    
    print(f"file size: {size_mb:.2f} MB")

# 
get_file_size("model_2024_hairstyle.tflite")

file size: 76.58 MB


In [10]:
# 80296224/1024/10244


##### Question 2

To be able to use this model, we need to know the index of the input and
the index of the output.

What's the output index for this model?

* 3
* 7
* 13 (correct)
* 24


In [11]:
import tensorflow.lite as tflite

interpreter = tflite.Interpreter(model_path="model_2024_hairstyle.tflite")
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

print(input_index, output_index)


0 13



##### Preparing the image
You'll need some code for downloading and resizing images. You can use
this code:


In [12]:

from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img


For that, you'll need to have `pillow` installed:
```bash
pip install pillow
```
Let's download and resize this image:

https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg

Based on the previous homework, what should be the target size for the image?

#### Question 3

Now we need to turn the image into numpy array and pre-process it.

> Tip: Check the previous homework. What was the pre-processing
> we did there?

After the pre-processing, what's the value in the first pixel, the R channel?

* 0.24(correct)
* 0.44
* 0.64
* 0.84


In [ ]:
pic_file=download_image("https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg")
pic_file=prepare_image(pic_file, (200, 200))
pic_file


In [14]:
import numpy as np
img_array = np.array(pic_file)

img_preprocessed = img_array / 255.0

first_pixel = img_preprocessed[0, 0]
r_value = first_pixel[0]  
g_value = first_pixel[1]   
b_value = first_pixel[2]  

print(f"First pixel RGB values:")
print(f"R: {r_value:.3f}")
print(f"G: {g_value:.3f}")
print(f"B: {b_value:.3f}")

First pixel RGB values:
R: 0.239
G: 0.408
B: 0.086



## Question 4

Now let's apply this model to this image. What's the output of the model?

* 0.293
* 0.493
* 0.693
* 0.893(correct)


In [15]:
X = img_preprocessed.reshape(1, 200, 200, 3)
interpreter.set_tensor(input_index, X.astype(np.float32))
interpreter.invoke()

prediction = interpreter.get_tensor(output_index)
print(f"Model prediction: {prediction[0][0]:.3f}")

Model prediction: 0.894


In [16]:
prediction

array([[0.8937741]], dtype=float32)


## Prepare the lambda code

Now you need to copy all the code into a separate python file. You will
need to use this file for the next two questions.

Tip: you can test this file locally with `ipython` or Jupyter Notebook
by importing the file and invoking the function from this file.

## Docker

For the next two questions, we'll use a Docker image that we already
prepared. This is the Dockerfile that we used for creating the image:

```docker
FROM public.ecr.aws/lambda/python:3.10

COPY model_2024_hairstyle_v2.tflite .

RUN pip install numpy==1.23.1
```

Note that it uses Python 3.10. The latest models of TF Lite
do not support Python 3.12 yet, so we need to use 3.10. Also,
for this part, we will use TensorFlow 2.14.0. We have tested
it, and the models created with 2.17 could be served with 2.14.0.

For that image, we also needed to use an older version of numpy
(1.23.1)

The docker image is published to [`agrigorev/model-2024-hairstyle:v3`](https://hub.docker.com/r/agrigorev/model-2024-hairstyle/tags).

A few notes:

* The image already contains a model and it's not the same model
  as the one we used for questions 1-4.
* The wheel for this combination that you'll need to use in your Docker image is https://github.com/alexeygrigorev/tflite-aws-lambda/raw/main/tflite/tflite_runtime-2.14.0-cp310-cp310-linux_x86_64.whl



#### Question 5

Download the base image `agrigorev/model-2024-hairstyle:v3`. You can do it with [`docker pull`](https://docs.docker.com/engine/reference/commandline/pull/).

So what's the size of this base image?

* 182 Mb
* 382 Mb
* 582 Mb
* 782 Mb (correct)

You can get this information when running `docker images` - it'll be in the "SIZE" column.



## Question 6

Now let's extend this docker image, install all the required libraries
and add the code for lambda.

You don't need to include the model in the image. It's already included.
The name of the file with the model is `model_2024_hairstyle_v2.tflite` and it's
in the current workdir in the image (see the Dockerfile above for the
reference).
The provided model requires the same preprocessing for images regarding target size and rescaling the value range than used in homework 8.

Now run the container locally.


What's the output from the model?

* 0.229
* 0.429 (correct)
* 0.629
* 0.829

